In [37]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
%cd /content/drive/MyDrive/Colab Notebooks/COSE474 TP

/content/drive/MyDrive/Colab Notebooks/COSE474 TP


# Import

In [ ]:
!pip install datasets
!pip install accelerate
!pip install evaluate

In [1]:
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm

from transformers import BertModel, BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

c:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
train_path = 'dataset/train_aug_backtranslate.csv'
test_path = 'dataset/test.csv'
dataset = load_dataset('csv', data_files={'train': train_path, 'test': test_path})

In [42]:
import gc
gc.collect()
torch.cuda.empty_cache()
del model

In [20]:
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=2).to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def tokenization(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenization, batched=True, remove_columns=["text"])
tokenized_dataset.set_format("torch")

Map: 100%|██████████| 2000/2000 [00:00<00:00, 2951.23 examples/s]


In [7]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
training_args = TrainingArguments(
    output_dir='./checkpoints/backtranslate_2',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    seed=42,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['train'],
    compute_metrics=compute_metrics,
)

NameError: name 'model' is not defined

In [24]:
trainer.train()

C:\Users\bestm\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 20%|██        | 625/3125 [19:49<1:21:23,  1.95s/it]

{'loss': 0.6652, 'learning_rate': 4e-05, 'epoch': 1.0}



 20%|██        | 625/3125 [30:17<1:21:23,  1.95s/it]

{'eval_loss': 0.6400505900382996, 'eval_accuracy': 0.633, 'eval_runtime': 628.788, 'eval_samples_per_second': 15.904, 'eval_steps_per_second': 0.994, 'epoch': 1.0}


 40%|████      | 1250/3125 [49:28<57:23,  1.84s/it]    

...

                                                     
100%|██████████| 3125/3125 [3:34:10<00:00,  5.16s/it]

{'eval_loss': 0.6899064183235168, 'eval_accuracy': 0.5086, 'eval_runtime': 1997.6839, 'eval_samples_per_second': 5.006, 'eval_steps_per_second': 0.313, 'epoch': 5.0}


100%|██████████| 3125/3125 [3:34:11<00:00,  4.11s/it]

{'train_runtime': 12851.5251, 'train_samples_per_second': 3.891, 'train_steps_per_second': 0.243, 'train_loss': 0.685969423828125, 'epoch': 5.0}


TrainOutput(global_step=3125, training_loss=0.685969423828125, metrics={'train_runtime': 12851.5251, 'train_samples_per_second': 3.891, 'train_steps_per_second': 0.243, 'train_loss': 0.685969423828125, 'epoch': 5.0})

# Load model

In [11]:
tokenized_dataset['test']['id']

tensor([   227,    301,    384,  ..., 199792, 199799, 199879])

In [39]:
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
ft_model = BertForSequenceClassification.from_pretrained(r"checkpoints\backtranslate\checkpoint-625", num_labels=2).to("cuda")

In [40]:
training_args = TrainingArguments(
    output_dir='./checkpoints/temp',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    seed=42,
    evaluation_strategy='no',
    logging_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=ft_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics
)

In [41]:
trainer.evaluate(eval_dataset=tokenized_dataset['test'])

100%|██████████| 125/125 [00:41<00:00,  3.01it/s]


{'eval_loss': 0.6163698434829712,
 'eval_accuracy': 0.6675,
 'eval_runtime': 42.046,
 'eval_samples_per_second': 47.567,
 'eval_steps_per_second': 2.973}

In [33]:
predictions = trainer.predict(test_dataset=tokenized_dataset['test'])
predictions = np.argmax(predictions.predictions, axis=-1)
predictions

100%|██████████| 125/125 [00:42<00:00,  2.97it/s]


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)